In [ ]:
# Reloads the code base
%load_ext autoreload

In [ ]:
# Reloads the code base
%autoreload 2

In [ ]:
import bittensor
import os
import torch
import torch.multiprocessing as mp 
import time
from loguru import logger
from termcolor import colored
import nest_asyncio 
nest_asyncio.apply()
logger.remove()

# **Init Bittensor** <a class="anchor" id="Init-Bittensor"></a>

## **Wallet** <a class="anchor" id="Wallet"></a>

<span style="color:green">**Description:**</span> The following cell creates your bittensor wallet. Your wallet holds two keys for you:

* <span style="color:blue">***coldkey***</span> (~/.bittensor/wallets/default/YOUR_WALLET_NAME):
    * Your coldkey is used to store, transfer, and stake tokens. It is "cold" because it is not loaded into the miner and remains encrypted on the device. 
<br />
<br />

* <span style="color:blue">***hotkey***</span> (~/.bittensor/wallets/YOUR_WALLET_NAME/hotkeys/YOUR_HOTKEY_NAME):  
    * Your hotkey is used by the miner to subscribe and set weights. It is "hot" because it is loaded into the running software (which can be insecure). It does not have permission to move funds.


<span style="color:red">**IMPORTANT**</span>

If you create keys ensure you store the generated mnemonic for **both** your hot and coldkey 
you will need these to recover your keys if you forget your password or lose access to this machine.



In [ ]:
# Fill in below to name your wallet and hotkey.
YOUR_WALLET_NAME = 'colab'
YOUR_HOTKEY_NAME = 'colab_hot'

# Fill in below if your need to regenerate your keys.
use_mnemonic = False # Set to true for key regeneration.
coldkey_mnemonic = "<to be filled>".split(' ')
hotkey_mnemonic = "<to be filled>".split(' ')

# Create the wallet object.
wallet = bittensor.Wallet(
    path = "~/.bittensor/wallets/",
    name = YOUR_WALLET_NAME,
    hotkey = YOUR_HOTKEY_NAME
)

# Optionally regens/creates your wallet keys.
if not wallet.has_coldkeypub:
    if use_mnemonic:
        wallet.regenerate_coldkey(mnemonic = coldkey_mnemonic, use_password = True)
    else:
        wallet.create_new_coldkey(n_words = 12, use_password = True )
if not wallet.has_hotkey:
    if use_mnemonic:
        wallet.regenerate_hotkey(mnemonic = hotkey_mnemonic)
    else:
        wallet.create_new_hotkey(n_words = 12)

# Assert before continuing
assert wallet.has_hotkey
assert wallet.has_coldkeypub

## **Config** <a class="anchor" id="Config"></a>

<span style="color:green">**Description:**</span> The following cells instantiate the Bittensor's background components through a config object. These are:

* <span style="color:blue">***subtensor***</span> (~/.bittensor/bittensor/subtensor.py):
    * A websocket connection to the blockchain; used to query chain state and make transactions.
<br />
* <span style="color:blue">***metagraph***</span> (~/.bittensor/bittensor/subtensor.py):
    * An object which maintains synced chain state as torch tensors.
<br />
* <span style="color:blue">***axon***</span> (~/.bittensor/bittensor/axon.py):
    * An RPC server which accepts requests from other peers in the network and makes those requests available to your miner.
<br />
* <span style="color:blue">***dendrite***</span> (~/.bittensor/bittensor/dendrite.py):
    * An RPC client which maintains an interface to other peers as a queryable, autograd-friendly torch.nn.Module.

<span style="color:red">**IMPORTANT:**</span> If bittensor is already initialized running these commands will recreate these components.


In [ ]:
# Uncomment to print all config items: 
# bittensor.help()
config = bittensor.Neuron.default_config()
config.neuron.multiprocessing = False
config.neuron.debug = True
config.receptor.do_backoff = False
config.receptor.timeout = 2
config.subtensor.network = 'kusanagi'

In [ ]:
# Init bittensor components
# NOTE: Running this call recreates bittensor objects.
bittensor.init( with_config = config, with_wallet = wallet )
print ( bittensor.Config.toString( bittensor.config ) )

## **Subscription** <a class="anchor" id="Subscription"></a>

<span style="color:green">**Description:**</span> Peers on the network advertise their existence by subscribing an RPC endpoint to the blockchain. <br />
Run the following cell to subscribe your endpoint; telling the others which ip/port you will be <br />
recieving queries on.

<span style="color:red">**IMPORTANT:**</span> Ensure you re-run this command if your endpoint changes.

In [ ]:
# Subscribe our endpoint to the chain.
bittensor.subtensor.subscribe(
    config.axon.external_ip, 
    config.axon.external_port,
    config.neuron.modality,
    wallet.coldkeypub,
    wait_for_finalization = True,
    timeout = 4 * bittensor.__blocktime__,
)

## **Metagraph** <a class="anchor" id="Metagraph"></a> 

<span style="color:green">**Description:**</span> The following cell syncs the latest chain-state into your metagraph


In [ ]:
# Syncs the latest chain information into the metagraph
# NOTE: this command is expensive.
bittensor.metagraph.sync()

In [ ]:
chain_block =  bittensor.metagraph.block()
uids_on_chain = bittensor.metagraph.uids()
n_neurons = torch.max(bittensor.metagraph.uids())
n_online = torch.numel(
  bittensor.metagraph.uids()[ 
    torch.where(
      bittensor.metagraph.block() - bittensor.metagraph.lastemit() < 100
    )
  ]
)
print ('The chain is at block: {}\n'.format(bittensor.metagraph.block()))
print ('There are {} neurons on the network\n'.format(n_neurons))
print ('These are their uids: {}\n'.format(bittensor.metagraph.uids()))
print ('{} neurons have emitted in the last 100 blocks\n'.format(n_online))
print ('{} Tao is staked'.format(torch.sum(bittensor.metagraph.S())))

In [ ]:
uid = bittensor.metagraph.uid_for_pubkey(wallet.hotkey.public_key)
index = bittensor.metagraph.index_for_uid(uid)
stake = bittensor.metagraph.S()[index]
rank = bittensor.metagraph.R()[index]
incentive = bittensor.metagraph.I()[index]
neuron = bittensor.metagraph.neurons()[index]
position = (torch.argsort(bittensor.metagraph.R(), dim=0) == index).nonzero(as_tuple=True)[0].item()
wallet_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
print ('Your network uid is {} for hotkey public key {}\n'.format(uid, wallet.hotkey.public_key))
print ('Your are subscribed at endpoint {}:{} for modality {}\n'.format(neuron.address, neuron.port, neuron.modality))
print ('You are staking \u03C4{} \n\nYou have a network rank of \u03C4{} for position {}/{}\n\nand are attaining incentive: \u03C4{}/block'.format(stake, rank, position, n_neurons, incentive))


# **Nucleus**  <a class="anchor" id="Nucleus"></a>

<span style="color:green">**Description:**</span> The following cell contains a bittensor ***Nucleus***: a custom machine learning model which can be served on the network. <br />
The model below uses a GPT2 kernel and learns from the AG-News dataset. ***Feel free to hack this part of the code.***

<span style="color:red">**IMPORTANT:**</span> Save this model between runs in-case your miner crashes. 

## **GPT2-Nucleus** <a class="anchor" id="GPT2-Nucleus"></a>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from types import SimpleNamespace
from transformers import GPT2Config, GPT2Model

class GPT2Nucleus(torch.nn.Module):
    # A simple as it gets Bittensor nucleus using a GPT2 kernel
    def __init__(self):
        super().__init__()
        huggingface_config = GPT2Config( vocab_size=bittensor.__vocab_size__, n_embd=bittensor.__network_dim__, n_layer=2, n_head=1, n_inner=8 )
        self.transformer = GPT2Model(huggingface_config)
        self.hidden_layer = nn.Linear( bittensor.__network_dim__, bittensor.__network_dim__ )
        self.target_layer = nn.Linear( bittensor.__network_dim__, bittensor.__vocab_size__, bias=False )
        self.loss_fct = nn.CrossEntropyLoss()
        
        # The scores you learn for other neurons in the network.
        # NOTE: This needs to be updated every time you re-sync your metagraph.
        self.row_weights = torch.rand( bittensor.metagraph.n(), requires_grad=True)

    def local_forward(self, inputs: torch.LongTensor):
        # Runs only the local part of the model.

        # To be filled.
        output = SimpleNamespace()

        # Apply our GPT transformer model.
        # local_context.shape = [ batch_size, sequence_len, network_dim ]
        output.local_context = self.transformer(input_ids=inputs, return_dict=True).last_hidden_state

        # Apply our dense layer and project it onto our target layer.
        # local_hidden.shape = [ batch_size, sequence_len, network_dim ]
        output.local_hidden = self.hidden_layer( output.local_context )

        # Project to our target dimension.
        # local_targets.shape = [ batch_size, sequence_len, vocab_size ]
        output.local_targets = self.target_layer( output.local_hidden )

        # Compute LM-loss 
        shift_targets = output.local_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.local_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output

    def remote_forward(self, inputs: torch.LongTensor, n_to_query:int = 10):
        # Runs the model with calls on the network.

        # Run the local model.
        output = self.local_forward( inputs )

        # Select peers to query based on weights + random pertubation.
        gamma = 0.9
        output.query_weights, output.query_indices = torch.topk(
            self.row_weights + torch.rand_like(self.row_weights) * gamma, 
            n_to_query
        )
        neurons_to_call = [ bittensor.metagraph.neurons()[idx] for idx in output.query_indices.tolist() ]
        inputs_to_send = [ inputs for _ in output.query_indices.tolist() ]
        output.query_uids = [bittensor.metagraph.uid_for_pubkey(neuron.public_key) for neuron in neurons_to_call]

        # Make network calls and then weight-join the responses.
        output.codes, responses = bittensor.forward_text( 
            neurons = neurons_to_call, 
            inputs = inputs_to_send
        )
        stacked_responses = torch.stack( responses, dim=2 )
        output.remote_context = torch.matmul( torch.transpose( stacked_responses, dim0=2, dim1=3), output.query_weights)

        # Compute the distillation loss between the local and remote context
        output.distillation_loss = F.mse_loss( output.local_context, output.remote_context.detach() )

        # Apply the hidden dense layer to the context.
        output.remote_hidden = self.hidden_layer( output.remote_context )

        # Project to our target dimension.
        output.remote_targets = self.target_layer( output.remote_hidden )

        # Compute our loss against our remote context.
        shift_targets = output.remote_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.remote_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output

In [ ]:
# Your nucleus
model = GPT2Nucleus()

## **Test-Nucleus** <a class="anchor" id="Test-Nucleus"></a>

In [ ]:
# Test remote forward call.
inputs = torch.tensor([bittensor.__tokenizer__()('the cat')['input_ids']])
output = model.remote_forward( inputs, n_to_query=20)
loss = output.local_loss + output.remote_loss + output.distillation_loss
loss.backward() #
print ('The distillation loss between your local and remote context is  {}\n'.format(output.distillation_loss))
print ('The loss with respect to your local context and the targets is {}\n'.format(output.local_loss))
print ('The loss with respect to your remote context and the targets is {}\n'.format(output.remote_loss))
print ('You queried {} remote neurons with\n\nuids\n {}\n\nresponse codes\n {}\n'.format(torch.numel(output.codes), output.query_uids, output.codes.tolist()))
print ('and gradients w.r.t your row weights\n', [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in output.query_indices.tolist()])


## Row-Weights

This model uses a parameter **row_weights** to select which peers to query during each call of model.remote_forward. <br>
The row_weights **are trainable with respect to the loss** because they are used to join these resonses like a typical Sparsely Gated Layer.
The miner uses these trained weights as proxy for the scores it sinks to the chain.



# **Training** <a class="anchor" id="Training"></a>

## **Training-Loop** <a class="anchor" id="Training-Loop"></a>

In [ ]:
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from loguru import logger
import random
from datasets import load_dataset
import time
import os

# ---- Dataset ---- 
dataset = load_dataset('ag_news')['train']
def nextbatch(data, batch_size, tokenizer):
    """ Returns a random batch of sentences from text dataset.
    """
    batch_text = []
    for _ in range(batch_size):
        batch_text.append(data[random.randint(0, len(data))]['text'])
    batch_inputs = tokenizer(batch_text, return_tensors='pt', padding=True, truncation=True)['input_ids']
    return batch_inputs

# --- Training Logger ----
logger.remove()
training_log_dir = os.path.expanduser('~/logs/training.log')
logger.add(training_log_dir, filter=lambda record: "training" in record["extra"], enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
def show_training_logs(length: int = 25):
    ! tail -n $length $training_log_dir

# ---- Tokenizer ----
# For encoding text inputs.
tokenizer = bittensor.__tokenizer__()

# ---- Optimizer ----
# For applying your gradient steps to the local model.
optimizer = torch.optim.SGD( model.parameters(), lr = 0.1, momentum = 0.99 )

# ---- Training Loop -----
def train( 
        stop_training: mp.Event,
    ):
    # ---- Loop until event is set ----
    training_step = 0
    batch_size = 5
    logger.bind(training=True).info('Loop starting... ')
    while not stop_training.is_set():
        try:
            optimizer.zero_grad() # Zeros out gradients for next accummulation

            # ---- Forward pass ----
            inputs = nextbatch( dataset, batch_size, tokenizer )
            outputs = model.remote_forward( inputs )

            # ---- Backward pass ----
            loss = outputs.local_loss + outputs.remote_loss + outputs.distillation_loss
            loss.backward() # Accumulates gradients on the model.
            clip_grad_norm_(model.parameters(), 0.8) # clip model gradients
            optimizer.step() # Applies accumulated gradients.

            # ---- Step logs ----
            logger.bind(training=True).info('->\nuids:{}\ncodes:{}\nweights:{}\ngrads:{}', 
                  outputs.query_uids, 
                  outputs.codes.tolist(), 
                  [float('{:0.3f}'.format(x)) for x in outputs.query_weights.tolist()],
                  [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in outputs.query_indices.tolist()])      
            logger.bind(training=True).info('gs:{} loss(local):{} loss(remote):{} loss(distill):{} dendrite:{}',
                  colored('{}'.format( training_step ), 'red'),
                  colored('{:.4f}'.format(outputs.local_loss.item()), 'green'),
                  colored('{:.4f}'.format(outputs.remote_loss.item()), 'blue'),
                  colored('{:.4f}'.format(outputs.distillation_loss.item()), 'red'),
                  bittensor.neuron.dendrite.toString())
            training_step += 1
        except Exception as e:
            logger.bind(training=True).exception("Training iteration exception.")
    logger.bind(training=True).complete()

## **Training Thread Runners** <a class="anchor" id="Training-Thread-Runners"></a>

In [ ]:
import threading
import torch.multiprocessing as mp 
import sys

join_timeout = 10

if 'quit_training' in locals():
    quit_training.set()
if 'training_thread' in locals() and training_thread.is_alive():
    training_thread.join( timeout = join_timeout )

quit_training = mp.Event()
training_thread = threading.Thread( target = train, args = (quit_training,),  name = 'training', daemon=True)

def stop_training():
    global quit_training
    global training_thread
    quit_training.set()
    if not training_thread.is_alive():
        return
    logger.bind(training=True).info("Joining...")
    training_thread.join( timeout = join_timeout )
    if not training_thread.is_alive():
        print ('Joined training thread',)
        logger.bind(training=True).info('Joined.')
    else:
        print ('Failed to join training thread')

def start_training():
    global quit_training
    global training_thread
    stop_training()
    quit_training = mp.Event()
    training_thread = threading.Thread( target = train, args = (quit_training,), name = 'training', daemon=True)
    training_thread.start()
    logger.bind(training=True).info("Started training.")
    print('new training thread:', training_thread)

In [ ]:
start_training()

In [ ]:
print (training_thread.is_alive())

In [ ]:
stop_training()

In [ ]:
show_training_logs(50)

# **Serving** <a class="anchor" id="Serving"></a>

In [ ]:
# Start the axon serving endpoint.
bittensor.axon.start()

## **Serving-Loop** <a class="anchor" id="Serving-Loop"></a>

In [ ]:
# ---- Serving logger ----
serving_log_dir = os.path.expanduser('~/logs/serving.log')
logger.add(serving_log_dir, filter=lambda record: "serving" in record["extra"], enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
def show_serving_logs(length: int = 25):
    ! tail -n $length $serving_log_dir

# ---- Serving loop -----
def serve ( 
  stop_serving: mp.Event,
):

    # ---- Loop until event is set -----
    serving_step = 0
    logger.bind(serving=True).info('Serving thread started: ')
    while not stop_serving.is_set():

        # ---- Pull request ----
        logger.bind(serving=True).info('Axon:{}, waiting for query ... ', bittensor.axon.toString())
        pong, pubkey, inputs, modality = bittensor.axon.next_forward_item( timeout = 10.0 )

        # ---- Process request ----
        if None not in [ pong, pubkey, inputs, modality]:
            logger.bind(serving=True).info('Recieved Query: from:{}, inputs.shape:{}', pubkey, inputs.shape)
            try:          
                outputs = model.local_forward( inputs ).local_hidden
                pong.send( outputs.detach() )
                logger.bind(serving=True).info('Sent response: to:{}, outputs.shape:{}', pubkey, outputs.shape)

            except Exception as e:
                logger.bind(serving=True).exception('Error in forward process with error {}', e)
                continue

      # ---- Tensorboard ----
      #bittensor.neuron.axon.toTensorboard(serving_tensorboard, serving_step)

## **Serving Thread Runners** <a class="anchor" id="Serving-Thread-Runners"></a>

In [ ]:
join_timeout = 10

if 'quit_serving' in locals():
    quit_serving.set()
if 'serving_thread' in locals() and serving_thread.is_alive():
    serving_thread.join( timeout = join_timeout )

quit_serving = mp.Event()
serving_thread = threading.Thread( target = serve, args = (quit_serving,),  name = 'serving', daemon=True)

def stop_serving():
    global quit_serving
    global serving_thread
    quit_serving.set()
    if serving_thread.is_alive():
        logger.bind(serving=True).info("Joining...")
        serving_thread.join( timeout = join_timeout )
    if not serving_thread.is_alive():
        print ('Joined serving thread',)
        logger.bind(serving=True).info('Joined.')
    else:
        print ('Failed to join serving thread')

def start_serving():
    global quit_serving
    global serving_thread
    stop_serving()
    quit_serving = mp.Event()
    serving_thread = threading.Thread( target = serve, args = (quit_serving,), name = 'serving', daemon=True)
    serving_thread.start()
    logger.bind(serving=True).info("Started serving.")
    print('new serving thread:', serving_thread)


In [ ]:
start_serving()

In [ ]:
print (serving_thread.is_alive())

In [ ]:
stop_serving()

In [ ]:
show_serving_logs(25)

## **Test-Serving** <a class="anchor" id="Test-Serving"></a>

In [ ]:
axon_endpoint = bittensor.proto.Neuron(
    address = bittensor.neuron.config.axon.external_ip,
    port = bittensor.neuron.config.axon.external_port,
    public_key = wallet.hotkey.public_key
)
start_time = time.time()
codes, responses = bittensor.forward_text( 
    neurons = [ axon_endpoint ],
    inputs = [ torch.tensor([[1]]) ]
)
end_time = time.time()
print(colored('Querying endpoint: {}:{}'.format(axon_endpoint.address, axon_endpoint.port), 'blue'))
if codes.item() == bittensor.proto.ReturnCode.Success:
    print(colored('Success', 'green'))
    print(colored('Response shape: {}'.format(responses[0].shape) , 'green'))
    print(colored('Query time: {}'.format(end_time - start_time) , 'green'))
else:
    print(colored('Failure with code: {}'.format(codes.item()), 'red'))
    print(colored('Ensure your axon is started with bittensor.axon.start()', 'red'))
    print(colored('Ensure your endpoint is accessable from the internet, perhaps behind your router\'s NAT?: {}', 'red'))


# **Weights**

## **Filtering Weights**

In [ ]:
# Get the trained weights from the chain.
weights_to_emit = model.row_weights.detach()

# Take topk
topk = 30
weights_to_emit, indices = torch.topk(weights_to_emit, topk)

# Normalize to 0,1
weights_to_emit = F.normalize(weights_to_emit, p = 1, dim = 0)

# Scatter back to size n.
weights_to_emit = torch.scatter(torch.zeros(bittensor.metagraph.n()), 0, indices, weights_to_emit)
print('Emitting weights: \n{}'.format(weights_to_emit))

## **Setting Weights**

In [ ]:
# Sets your incentive weights on the chain.
bittensor.metagraph.set_weights( weights = weights_to_emit )

In [ ]:
your_uid = bittensor.metagraph.uids()[0]
print ('Your weights (encoded as uint32s) on the chain are: \n\n {} \n'.format(bittensor.subtensor.weight_vals_for_uid( your_uid )))

print ('For uids \n {}'.format(bittensor.subtensor.weight_uids_for_uid( your_uid )))

# **Transactions** <a class="anchor" id="Transactions"></a>

## **Unstaking-Funds** <a class="anchor" id="Unstaking-Funds"></a>

In [ ]:
from bittensor.utils.balance import Balance
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Unstake: {} Tao from hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = bittensor.subtensor.unstake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
    new_stake = bittensor.subtensor.get_stake_for_uid( bittensor.metagraph.uid_for_pubkey(wallet.hotkey.public_key) )
    print(colored("Unstaked: {} Tao from hotkey: {} to coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))
else:
    print(colored("Unstaking transaction failed", 'red'))

## **Staking-Funds** <a class="anchor" id="Staking-Funds"></a>

In [ ]:
from bittensor.utils.balance import Balance 
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Stake: {} Tao to hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = bittensor.subtensor.add_stake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
  new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
  new_stake = bittensor.subtensor.get_stake_for_uid(bittensor.metagraph.uid_for_pubkey(wallet.hotkey.public_key))
  print(colored("Staked: {} Tao to hotkey: {} from coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
  print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
  print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))

else:
  print(colored("Staking transaction failed", 'red'))

## **Transfering-Funds** <a class="anchor" id="Transfering-Funds"></a>

In [ ]:
amount = 0.01
destination_public_key = wallet.coldkey.public_key
amount = Balance.from_float( amount )
balance = bittensor.subtensor.get_balance( wallet.coldkey.public_key )
if balance < amount:
    print(colored("Not enough balance ({}) to transfer {}".format(balance, amount), 'red'))
    quit()

print(colored("Requesting transfer of {} Tao, from coldkey.pub: {} to dest.pub: {}".format(amount.tao, wallet.coldkey.public_key, destination_public_key), 'blue'))
print("Waiting for finalization...",)
result = bittensor.subtensor.transfer(destination_public_key, amount, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    print(colored("Transfer finalized with amount: {} Tao to dest: {} from coldkey.pub: {}".format(amount.tao, destination_public_key, wallet.coldkey.public_key), 'green'))
    new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
    destination_balance = bittensor.subtensor.get_balance(destination_public_key)
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("The destination has new balance: {} Tao".format( new_balance.tao ) , 'green'))
else:
    print(colored("Transfer failed", 'red'))